In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
X_train = X_train.reshape(-1,28,28,1).astype('float32') /255.0
X_test = X_test.reshape(-1,28,28,1).astype('float32') /255.0

In [7]:
model = keras.Sequential (
    [
        layers.Input(shape = (28,28,1)),
        layers.Conv2D(64,3,padding ='same'),
        layers.ReLU(),
        layers.Conv2D(128,3,padding ='same'),
        layers.ReLU(),
        layers.Flatten(),
        layers.Dense(10),

    ],
    name ='model',
)

compile: Set up the optimizer and loss function.

train_step: Perform a single training step, calculating gradients and updating the model's weights.

test_step: Evaluate the model on a batch of test data.

In [8]:
class CustomFit(keras.Model):
    def __init__(self, model):
        super(CustomFit, self).__init__()
        self.model = model

    def compile(self, optimizer, loss):
        super(CustomFit, self).compile()
        self.optimizer = optimizer
        self.loss = loss

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            # Caclulate predictions
            y_pred = self.model(x, training=True)

            # Loss
            loss = self.loss(y, y_pred)

        # Gradients
        training_vars = self.trainable_variables
        gradients = tape.gradient(loss, training_vars)

        # Step with optimizer
        self.optimizer.apply_gradients(zip(gradients, training_vars))
        acc_metric.update_state(y, y_pred)

        return {"loss": loss, "accuracy": acc_metric.result()}

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_pred = self.model(x, training=False)

        # Updates the metrics tracking the loss
        loss = self.loss(y, y_pred)

        # Update the metrics.
        acc_metric.update_state(y, y_pred)
        return {"loss": loss, "accuracy": acc_metric.result()}


acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")

training = CustomFit(model)
training.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

training.fit(X_train, y_train, batch_size=64, epochs=2)
training.evaluate(X_test, y_test, batch_size=64)

Epoch 1/2
157/157 [==============================] - 1s 4ms/step - loss: 0.0503 - accuracy: 0.9666


[0.9674000144004822, 0.0010196175426244736]